In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import datetime

path = '/Users/luis-felipe/ExamenMetodo/djangoMetodo/app/static/'
df1 = pd.read_excel(path + 'ED_TASAJ.xlsx').set_index('Descripción series').drop('Reg',axis=1)
df2 = pd.read_excel(path + 'EST_GEN_DEU_01.xlsx').set_index('Descripción series').drop('Reg',axis=1)
df3 = pd.read_excel(path + 'ED_VAR_REM_M_2023_EMP.xlsx').set_index('Descripción series').drop('Reg',axis=1)
df4 = pd.read_excel(path + 'EXE_EOF_1.xlsx').set_index('Descripción series').drop('Reg',axis=1)

df = pd.concat([df1,df2,df3,df4],join='inner').drop('\xa0\xa0\xa0Hombres',axis=0).drop('\xa0\xa0\xa0Mujeres',axis=0)
indices = df.index.tolist()
valores = ["TasaDesempleo", "DeudasTotal","VariacionDeSueldo","GastoDeEmpleo","Inflacion"]

df.columns = pd.to_datetime(df.columns)

columnasNuevas ={}
for i in range(len(indices)):
    columnasNuevas[indices[i]]=valores[i]
df = df.rename(index=columnasNuevas)

df1 = df.T




experimentos = 1000
mes = 12
dias = [31,28,31,30,31,30,31,31,30,31,30,31]

df2 = pd.DataFrame(df1["Inflacion"])
df2["mes"] = df1.index.month
df2["cambio_pct"] = df2["Inflacion"].pct_change() * 100  # Calcular cambio porcentual

# Agrupar los cambios porcentuales por mes
meses = df2.groupby("mes").agg({"cambio_pct": list})

inflacionInicial = df1['Inflacion'].values.tolist()[-1]
x = list(range(1, sum(dias)+2))
print(len(x))
y = []
context = {
    'x': x
}
for i in range(experimentos):
    context['recta_' + str(i+1)] = []

data = pd.DataFrame(columns=x, index=list(context.keys())[1:])
for i in range(experimentos):
    inflacion = [inflacionInicial]
    for k in range(1, mes+1):
        for l in range(dias[k-1]):
                # Usar cambios porcentuales históricos
            cambio = np.random.choice([x for x in meses.loc[k, "cambio_pct"] if not np.isnan(x) and not np.isinf(x)])
            nuevoValor = inflacion[-1] * (1 + cambio/100/30)  # Aplicar cambio porcentual diario
            inflacion.append(float(nuevoValor))
    print(len(inflacion))
    data.iloc[i] = inflacion

data.to_excel(path + 'simulacion_inflacion.xlsx')


366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366
366


In [6]:
path = '/Users/luis-felipe/ExamenMetodo/djangoMetodo/app/static/'
df1 = pd.read_excel(path + 'ED_TASAJ.xlsx').set_index('Descripción series').drop('Reg',axis=1)
df2 = pd.read_excel(path + 'EST_GEN_DEU_01.xlsx').set_index('Descripción series').drop('Reg',axis=1)
df3 = pd.read_excel(path + 'ED_VAR_REM_M_2023_EMP.xlsx').set_index('Descripción series').drop('Reg',axis=1)
df4 = pd.read_excel(path + 'EXE_EOF_1.xlsx').set_index('Descripción series').drop('Reg',axis=1)

df = pd.concat([df1,df2,df3,df4],join='inner').drop('\xa0\xa0\xa0Hombres',axis=0).drop('\xa0\xa0\xa0Mujeres',axis=0)
indices = df.index.tolist()
valores = ["TasaDesempleo", "DeudasTotal","VariacionDeSueldo","GastoDeEmpleo","Inflacion"]

df.columns = pd.to_datetime(df.columns)

columnasNuevas ={}
for i in range(len(indices)):
    columnasNuevas[indices[i]]=valores[i]
df = df.rename(index=columnasNuevas)

df1 = df.T




experimentos = 1000
mes = 12
dias = [31,28,31,30,31,30,31,31,30,31,30,31]

# Calcular cambio porcentual




x = list(range(1, sum(dias)+2))
print(len(x))
y = []
context = {
    'x': x
}
for i in range(experimentos):
    context['recta_' + str(i+1)] = []

data = pd.DataFrame(columns=x, index=list(context.keys())[1:])


df3 = pd.DataFrame(df1["TasaDesempleo"])
df3["mes"] = df1.index.month
df3["cambio_pct"] = df3["TasaDesempleo"].pct_change() * 100  # Calcular cambio porcentual

# Agrupar los cambios porcentuales por mes
meses = df3.groupby("mes").agg({"cambio_pct": list})

desempleoInicial = df1['TasaDesempleo'].values.tolist()[-1]

for i in range(experimentos):
    desempleo = [desempleoInicial]
    for k in range(1, mes+1):
        for l in range(dias[k-1]):
            # Usar cambios porcentuales históricos
            cambio = np.random.choice([x for x in meses.loc[k, "cambio_pct"] if not np.isnan(x)])
            nuevoValor = desempleo[-1] * (1 + cambio/100/30)  # Aplicar cambio porcentual diario
            desempleo.append(nuevoValor)
    data.iloc[i] = desempleo
data.to_excel(path + 'simulacion_desempleo.xlsx')
    



366


30.416666666666668